# Testing the CurrenSee Secrets Management

This notebook tests the new secrets management system that integrates with Google Secret Manager.

The system allows secrets to be retrieved from:
1. Google Secret Manager (for production)
2. Environment variables (as fallback for local development)

In [1]:
# Add the project root to the Python path
import sys
import os
from pathlib import Path

# Add project root to path to allow importing currensee modules
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Verify the path
print(f"Project root: {project_root}")
print(f"Python path: {sys.path}")

Project root: /home/apavlovic/capstone/currensee
Python path: ['/opt/conda/envs/ap_currensee_v2/lib/python311.zip', '/opt/conda/envs/ap_currensee_v2/lib/python3.11', '/opt/conda/envs/ap_currensee_v2/lib/python3.11/lib-dynload', '', '/home/apavlovic/.local/lib/python3.11/site-packages', '/opt/conda/envs/ap_currensee_v2/lib/python3.11/site-packages', '/home/jupyter/ap_currensee/currensee', '/home/apavlovic/capstone/currensee']


## 1. Directly Testing the SecretManager Module

First, let's test the raw `secrets.py` module directly.

In [2]:
# Import the secrets module
from currensee.core.secrets import get_secret, get_secret_str, secret_manager

# Check if we can initialize
print(f"Secret Manager initialized with project_id: {secret_manager._project_id}")

2025-05-18 06:58:01,530 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


Secret Manager initialized with project_id: adsp-34002-on02-sopho-scribe


In [3]:
# Test retrieving a secret from Google Secret Manager
serper_key = get_secret("SERPER_API_KEY")

if serper_key:
    print(f"Successfully retrieved SERPER_API_KEY: {serper_key[:5]}...")
else:
    print("Failed to retrieve SERPER_API_KEY")

# Test retrieving as SecretStr
serper_secret = get_secret_str("SERPER_API_KEY")
if serper_secret:
    print(f"Successfully retrieved SERPER_API_KEY as SecretStr: {serper_secret.get_secret_value()[:5]}...")
else:
    print("Failed to retrieve SERPER_API_KEY as SecretStr")

2025-05-18 06:58:02,531 - currensee.core.secrets - INFO - Initializing Secret Manager client...
2025-05-18 06:58:02,575 - currensee.core.secrets - INFO - Secret Manager client initialized successfully
2025-05-18 06:58:02,577 - currensee.core.secrets - INFO - Attempting to retrieve SERPER_API_KEY from Google Secret Manager
2025-05-18 06:58:02,660 - currensee.core.secrets - INFO - Successfully retrieved SERPER_API_KEY from Google Secret Manager


Successfully retrieved SERPER_API_KEY: 22c92...
Successfully retrieved SERPER_API_KEY as SecretStr: 22c92...


## 2. Testing Integration with Settings

Next, let's check that our settings module correctly integrates with the secrets manager.

Note: This requires updating the settings.py file as described in the documentation.

In [ ]:
# Import settings
from currensee.core.settings import settings

# Check if SERPER_API_KEY is loaded in settings
if hasattr(settings, 'SERPER_API_KEY') and settings.SERPER_API_KEY:
    print(f"SERPER_API_KEY loaded in settings: {settings.SERPER_API_KEY.get_secret_value()[:5]}...")
else:
    print("SERPER_API_KEY not configured in settings")

## 3. Testing with Non-Existent Secret

The system should gracefully handle requests for non-existent secrets.

In [ ]:
# Try to get a non-existent secret
non_existent = get_secret("NON_EXISTENT_SECRET_KEY")
print(f"Result for non-existent secret: {non_existent}")

# This should return None without raising an exception

In [ ]:
get_secret_str("SERPER_API_KEY")